In [12]:
import pandas as pd
import os
import glob
import io 
import numpy as np
import matplotlib.pyplot as plt
import nibabel as nib
import seaborn as sns
from scipy.stats import zscore
from brainspace.utils.parcellation import reduce_by_labels, map_to_mask
from brainspace.gradient import GradientMaps
from brainspace.datasets import load_parcellation
from brainspace.plotting import plot_hemispheres
from brainspace.utils.parcellation import map_to_labels
from sklearn.metrics.pairwise import cosine_similarity
import dill

emb_dir ='/home/dimuthu1/scratch/PPMI_project2/derivatives/analysis/smoothed/gradients/bs_emb'

aligned_emb_ctx_R_12 = dill.load(open(emb_dir+"/aligned_emb_ctx_month12.pickle", "rb"))
aligned_emb_ctx_R_24 = dill.load(open(emb_dir+"/aligned_emb_ctx_month24.pickle", "rb"))
#emb = np.load(emb_dir+"/emb_ctx_12.pickle")

In [13]:
ngradients = 4
labeling = nib.load('../cfg/Schaefer2018_1000Parcels_7Networks_order.dlabel.nii').get_fdata()
#print(np.shape(labeling[1]))
mask = ~np.isin(labeling[0],0)

labeling = labeling.astype(int)

#labeling = np.squeeze(labeling).shape
#print(np.shape(labeling))

df = pd.read_table('../participants.tsv')
subjects = df.participant_id.to_list() 
subj = [ s.strip('sub-') for s in subjects ]

nsubjects = len(subj)


pixdim[1,2,3] should be non-zero; setting 0 dims to 1


In [10]:
 def project_to_cortex(emb_ctx,session,subject):
    grad = [None] * 4
    for i, g in enumerate(emb_ctx.T):
        grad[i] = map_to_labels(g, labeling[0], mask=mask, fill=np.nan)

    gii = nib.gifti.GiftiImage()

    for g in range(0,len(grad)):
        gii.add_gifti_data_array(
            nib.gifti.GiftiDataArray(data=grad[g][:32492].astype(np.float32)) # For left hemisphere, right hemisphere change to [32492:]
        )
    nib.save(gii, emb_dir+'/mean_ctx_L_gradients_'+session+'_'+subject+'.func.gii')

    for i, g in enumerate(emb_ctx.T):
        grad[i] = map_to_labels(g, labeling[0], mask=mask, fill=np.nan)

    gii = nib.gifti.GiftiImage()

    for g in range(0,len(grad)):
        gii.add_gifti_data_array(
            nib.gifti.GiftiDataArray(data=grad[g][32492:].astype(np.float32)) # For left hemisphere, right hemisphere change to [32492:]
        )
    nib.save(gii, emb_dir+'/mean_ctx_R_gradients_'+session+'_'+subject+'.func.gii')

In [4]:

#sliing the 3D matrix with aligned data for all the subjects and projecting the gradients to cortex


for i, s in enumerate(subj):
    emb_ctx = aligned_emb_ctx_R_12[:,:,i]
    project_to_cortex(emb_ctx,"month12",s)

In [14]:
aligned_emb_sbctx_L_12 = dill.load(open(emb_dir+"/aligned_emb_sbctx_L_month12.pickle", "rb"))
aligned_emb_sbctx_L_24 = dill.load(open(emb_dir+"/aligned_emb_sbctx_L_month24.pickle", "rb"))

aligned_emb_sbctx_R_12 = dill.load(open(emb_dir+"/aligned_emb_sbctx_R_month12.pickle", "rb"))
aligned_emb_sbctx_R_24 = dill.load(open(emb_dir+"/aligned_emb_sbctx_R_month24.pickle", "rb"))

In [21]:
out_path='/home/dimuthu1/scratch/PPMI_project2/derivatives/analysis/smoothed/plots/sbctx_nii'

def get_sbctx_projections(grad_array,session, grad_no,subj):
    atlas = nib.load('/home/dimuthu1/scratch/PPMI_project2/PPMI_gradients/cfg/91282_Greyordinates.dscalar.nii').get_fdata()
    img = nib.load('/home/dimuthu1/scratch/PPMI_project2/PPMI_gradients/cfg/91282_Greyordinates.dscalar.nii')

    true_vals = atlas[0]
    put_L = np.where(true_vals==12)
    put_R = np.where(true_vals==51)
    cau_L = np.where(true_vals==11)
    cau_R = np.where(true_vals==50)
    acc_L = np.where(true_vals==26)
    acc_R = np.where(true_vals==58)

    #subctx_vals = np.concatenate((put_L, cau_L, acc_L),axis=1)
    subctx_vals = np.concatenate((put_R, cau_R, acc_R),axis=1)
    print(np.shape(subctx_vals[0,:]))
    print(np.shape(grad_array))

    sbctx_ind = subctx_vals[0,:]
    for i, val  in enumerate(sbctx_ind):
        true_vals[val]= grad_array[i]*1000+1000
        

    for i, val in enumerate(true_vals):
        #print(i)
        if i not in sbctx_ind.tolist():
            true_vals[i]= 0 #float("nan")

    true_vals = true_vals.reshape(1,-1)
    new_img = nib.Cifti2Image(true_vals, img.header)
    nib.save(new_img, out_path+'/'+subj+'-'+session+'_sbctx_R_'+grad_no+'_image.dscalar.nii')


In [22]:
for i, s in enumerate(subj):
    emb_sbctx_12 = aligned_emb_sbctx_R_12[:,2,i] #projecting gradient 3
    #print(np.shape(emb_sbctx_12))
    get_sbctx_projections(emb_sbctx_12,'m12','grad_3',s)

    emb_sbctx_24 = aligned_emb_sbctx_R_24[:,2,i] #projecting gradient 3
    #print(np.shape(emb_sbctx_24))
    get_sbctx_projections(emb_sbctx_24,'m24','grad_3',s)

(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)
(1905,)


In [19]:
emb_sbctx_L_12 = dill.load(open(emb_dir+"/emb_sbctx_L_month12.pickle", "rb"))
emb_sbctx_L_24 = dill.load(open(emb_dir+"/emb_sbctx_L_month24.pickle", "rb"))

emb_sbctx_R_12 = dill.load(open(emb_dir+"/emb_sbctx_R_month12.pickle", "rb"))
emb_sbctx_R_24 = dill.load(open(emb_dir+"/emb_sbctx_R_month24.pickle", "rb"))

emb_sbctx_R_12 = emb_sbctx_R_12.gradients_[:,0]  #Mean gradient is passed as an object
#print(np.shape(emb_sbctx_R_12.gradients_))
get_sbctx_projections(emb_sbctx_R_12,'m12','grad_1','mean')

[87736 87737 87738 ... 59684 59685 59686]


In [36]:
#Run cifti_2_nii.sh before running this block

from nilearn.image import concat_imgs

m12 = []
m24 = []

for i, s in enumerate(subj):

    m12.append(glob.glob(out_path+'/'+s+'-m12_sbctx_R_grad_3_3d.nii')[0])
    m24.append(glob.glob(out_path+'/'+s+'-m24_sbctx_R_grad_3_3d.nii')[0])


all_nii = m12 + m24
print(np.shape(all_nii))

image_4D = concat_imgs(all_nii)

nib.save(image_4D, out_path+'/concatednated_4D.nii')

#result_img = smooth_img(out_path+'/*3d.nii') 

(74,)


In [23]:
from nilearn.image import concat_imgs
diff_files = []
for i, s in enumerate(subj):

    diff_files.append(glob.glob(out_path+'/'+s+'_diff.nii.gz')[0])

#print(diff_files)

image_diff_4D = concat_imgs(diff_files)
nib.save(image_diff_4D, out_path+'/concatednated_diff_4D.nii')



